In [3]:
import os
import numpy as np
import pandas as pd
from aeon.datasets import load_classification

from utils.tools import create_directory
from utils.constants import datasets

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


def fit_classifier(all_labels, X_train, y_train, X_val=None, y_val=None, epochs=10, batch_size=16):
    nb_classes = len(np.unique(all_labels))
    # Create Classifier --------------------------------------------------------
    if classifier_name == "FCN" or classifier_name == "ResNet":
        input_shape = (X_train.shape[1], X_train.shape[2])
    elif classifier_name == "lstm_dcnn" or classifier_name == "MLSTM_FCN":
        input_shape = (X_train.shape[1], X_train.shape[2])
    else:
        input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
    # Call Classifier ----------------------------------------------------------
    classifier = create_classifier(classifier_name, input_shape, nb_classes, verbose=True)
    # Train Classifier ----------------------------------------------------------
    if X_val is None:
        classifier.fit(X_train, y_train, None, None, epochs, batch_size)
    else:
        classifier.fit(X_train, y_train, X_val, y_val, epochs, batch_size)
    return classifier


def create_classifier(classifier_name, input_shape, nb_classes, verbose=False):
    if classifier_name == "Disjoint_CNN":
        from classifiers import Disjoint_CNN
        return Disjoint_CNN.Classifier_Disjoint_CNN(sub_output_directory, input_shape, nb_classes, verbose)


# Problem Setting -----------------------------------------------------------------------------------------------------
ALL_Results = pd.DataFrame()
ALL_Results_list = []
problem_index = 0
data_path = os.getcwd() + '/multivariate_ts/'
# Hyper-Parameter Setting ----------------------------------------------------------------------------------------------
classifier_name = "Disjoint_CNN"  # Choose the classifier name from aforementioned List
epochs = 500
Resample = 1  # Set to '1' for default Train and Test Sets, and '30' for running on all resampling
# ----------------------------------------------------------------------------------------------------------------------

for problem in datasets[3:5]:
    # Load Data --------------------------------------------------------------------------------------------------------
    output_directory = os.getcwd() + '/Results_' + classifier_name + '/' + problem + '/'
    create_directory(output_directory)
    print("[Main] Problem: {}".format(problem))
    itr_result = [problem]
    # load --------------------------------------------------------------------------
    # set data folder
    X_train, Y_train = load_classification(problem,  split="train")
    X_test, Y_test = load_classification(problem,  split="test")

    all_data = np.vstack((X_train, X_test))
    all_labels = np.hstack((Y_train, Y_test))
    all_indices = np.arange(len(all_data))

    sub_output_directory = output_directory + str(1) + '/'
    create_directory(sub_output_directory)
    # Default Train and Test Set
    x_train = X_train
    x_test = X_test
    y_train = Y_train
    y_test = Y_test
    
    # Making Consistent with Keras Output -------------------------------------------------
    all_labels_new = np.concatenate((y_train, y_test), axis=0)
    print("[Main] All labels: {}".format(np.unique(all_labels_new)))
    tmp = pd.get_dummies(all_labels_new).values
    y_train = tmp[:len(y_train)]
    y_test = tmp[len(y_train):]

    # Making Consistent with Keras Input ---------------------------------------------------
    if classifier_name == "FCN" or classifier_name == "ResNet" or classifier_name == "MLSTM_FCN":
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[2], x_train.shape[1])
        x_test = x_test.reshape(x_test.shape[0], x_test.shape[2], x_test.shape[1])
    else:
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
        x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

    # classifier-----------------------------------------------------------------
    # Dynamic Batch-size base on Data
    if 'EigenWorms' in problem or 'DuckDuck' in problem:
        batch_size = 1
    else:
        # batch_size = np.ceil(x_train.shape[0] / (8 * (np.max(y_train.shape[1]) + 1)))
        batch_size = 8
        
    val_index = np.random.randint(0, np.int(x_train.shape[0]), np.int(x_train.shape[0] / 10), dtype=int)
    x_val = x_train[val_index, :]
    y_val = y_train[val_index, :]

    classifier = fit_classifier(all_labels_new, x_train, y_train, x_val, y_val, epochs, batch_size)
    metrics_test, conf_mat = classifier.predict(x_test, y_test, best=True)
    metrics_test2, conf_mat2 = classifier.predict(x_test, y_test, best=False)

    metrics_test['train/val/test/test2'] = 'test'
    metrics_test2['train/val/test/test2'] = 'test2'
    metrics = pd.concat([metrics_test, metrics_test2]).reset_index(drop=True)

    print("[Main] Problem: {}".format(problem))
    print(metrics.head())

    metrics.to_csv(sub_output_directory + 'classification_metrics.csv')
    np.savetxt(sub_output_directory + 'confusion_matrix.csv', conf_mat, delimiter=",")
    itr_result.append(metrics.accuracy[0])
    itr_result.append(metrics.accuracy[1])
    sub_output_directory = []

    if len(ALL_Results_list) == 0:
        ALL_Results_list = np.hstack((ALL_Results_list, itr_result))
    else:
        ALL_Results_list = np.vstack((ALL_Results_list, itr_result))

    problem_index = problem_index + 1

ALL_Results = pd.DataFrame(ALL_Results_list)
ALL_Results.to_csv(os.getcwd() + '/Results_' + classifier_name + '/'+'All_results1.csv')


[Main] Problem: DuckDuckGeese
[Main] All labels: ['black-bellied_whistling_duck' 'canadian_goose' 'greylag_goose'
 'pink-footed_goose' 'white-faced_whistling_duck']
Creating Disjoint_CNN Classifier


/tmp/ipykernel_1054311/1751919737.py:95: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  val_index = np.random.randint(0, np.int(x_train.shape[0]), np.int(x_train.shape[0] / 10), dtype=int)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1345, 270, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 1345, 270, 64)  │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 1345, 270, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ elu_8 (ELU)                     │ (None, 1345, 270, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 1345, 1, 64)    │     1,105,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 1345, 1, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ elu_9 (ELU)                     │ (None, 1345, 1, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ permute_3 (Permute)             │ (None, 1345, 64, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 1345, 64, 64)   │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 1345, 64, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ elu_10 (ELU)                    │ (None, 1345, 64, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 1345, 1, 64)    │       262,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 1345, 1, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ elu_11 (ELU)                    │ (None, 1345, 1, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ permute_4 (Permute)             │ (None, 1345, 64, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 1345, 1, 64)    │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 1345, 1, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ elu_13 (ELU)                    │ (None, 1345, 1, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ permute_5 (Permute)             │ (None, 1345, 64, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 1345, 64, 64)   │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 1345, 64, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ elu_14 (ELU)                    │ (None, 1345, 64, 64)   │             

 Total params: 1,646,533 (6.28 MB)

 Trainable params: 1,645,637 (6.28 MB)

 Non-trainable params: 896 (3.50 KB)

[Disjoint_CNN] Training Disjoint_CNN Classifier


2024-03-18 14:02:23.911092: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[64,64,1,270]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,1,1345,270]{3,2,1,0}, f32[64,1,1345,1]{3,2,1,0}), window={size=1345x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-03-18 14:02:23.967717: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.056723509s
Trying algorithm eng0{} for conv (f32[64,64,1,270]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,1,1345,270]{3,2,1,0}, f32[64,1,1345,1]{3,2,1,0}), window={size=1345x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_

In [11]:
datasets = ["ArticularyWordRecognition","AtrialFibrillation","BasicMotions","Cricket",
            "DuckDuckGeese","EigenWorms", "Epilepsy","ERing","EthanolConcentration","FaceDetection","FingerMovements",
            "HandMovementDirection","Handwriting","Heartbeat",
            "Libras","LSST","MotorImagery","NATOPS","PEMS-SF","PenDigits","PhonemeSpectra",
            "RacketSports", "SelfRegulationSCP1","SelfRegulationSCP2",
            "StandWalkJump","UWaveGestureLibrary"]

for i in range(len(datasets)):
    problem = datasets[i]
    try:
        X_train, Y_train = load_classification(problem,  split="train")
        X_test, Y_test = load_classification(problem,  split="test")
        print("dataset",problem)
        print("X_train.shape"," X_test.shape")
        print(np.array(X_train).shape, np.array(X_test).shape)
    except Exception as e:
        print("Error: ",e)
        print("error index: ", i+1)

dataset ArticularyWordRecognition
X_train.shape  X_test.shape
(275, 9, 144) (300, 9, 144)
dataset AtrialFibrillation
X_train.shape  X_test.shape
(15, 2, 640) (15, 2, 640)
dataset BasicMotions
X_train.shape  X_test.shape
(40, 6, 100) (40, 6, 100)
dataset Cricket
X_train.shape  X_test.shape
(108, 6, 1197) (72, 6, 1197)
dataset DuckDuckGeese
X_train.shape  X_test.shape
(50, 1345, 270) (50, 1345, 270)
dataset EigenWorms
X_train.shape  X_test.shape
(128, 6, 17984) (131, 6, 17984)
dataset Epilepsy
X_train.shape  X_test.shape
(137, 3, 206) (138, 3, 206)
dataset ERing
X_train.shape  X_test.shape
(30, 4, 65) (270, 4, 65)
dataset EthanolConcentration
X_train.shape  X_test.shape
(261, 3, 1751) (263, 3, 1751)
dataset FaceDetection
X_train.shape  X_test.shape
(5890, 144, 62) (3524, 144, 62)
dataset FingerMovements
X_train.shape  X_test.shape
(316, 28, 50) (100, 28, 50)
dataset HandMovementDirection
X_train.shape  X_test.shape
(160, 10, 400) (74, 10, 400)
dataset Handwriting
X_train.shape  X_test.sh

In [1]:
number = 5.215406062575312 * 10**-8
print(number)

5.215406062575312e-08
